In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.device("cuda")

In [2]:
import sys
sys.path.append('./huggingface_models/')
sys.path.append('./utils/')
from sample_utils import *
from inference_utils import *
from codenet_process_utils import *
from self_training_utils import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
with open(cached_path + 'codenet_merged_filtered_dict.json') as infile:
    merged_filtered_dict = json.load(infile)

In [5]:
for lang in new_langs:
    print(lang, len(merged_filtered_dict[lang]))

C++ 37126
Java 10511
Python 26911
C# 3101
C 12424


In [ ]:
# call_dict
# preds_lang_dict
# filtered_dict
# 

### Show Graph

In [ ]:
lang = "Java"
key = 9981 #9981
code_dic = merged_filtered_dict[lang][key]['code_dic']
program = code_dic['program_formatted']
paras = code_dic['parameter_lists']
return_types = code_dic['return_types']
function_names = code_dic['function_names']
functions = code_dic['functions']
function = "\n".join(functions)
pieces = code_dic['program_pieces']
piece = "".join(pieces)
target_call = code_dic['target_call']
# print(program)
print(function)
print(piece)
print(function_names)
print(return_types)
print(paras)
print(target_call)

In [ ]:
lang1 = "Java"
root1, graph1, graph_pruned1, graph_sibs1, graph_pruned_sibs1 = pipeline(code1, ast_parsers[lang1], lang1)
# root1, graph1, graph_pruned1, graph_sibs1, graph_pruned_sibs1 = refine_graphs(root1, graph_pruned_sibs1)
show_graph(root1, graph1)

### Preprocess data for model training

#### No tokenization preprocessing

In [4]:
# processing steps:
# remove comments, empty lines format_codestring_codenet(codestring, lang)
# replace new_lines notok_prepro(codestring, lang, is_plbart)
# after decoding, do notok_detok notok_detok(codestring, lang, is_plbart)
# do detok_format(codestring, detokenizer) to get detokenized version for Java and Python

is_plbart = True
# merged_filtered_dict = get_prepro_filtered_dict(merged_filtered_dict, is_plbart)
merged_filtered_dict = get_prepro_filtered_dict(None, is_plbart)

#### Split into batches for self-training

In [15]:
# Java-Python
# 3 batches 
num_batchs = 3
batch_split_dict = {}
for lang in new_langs:
    length = len(merged_filtered_dict[lang])
    batch_size = length//num_batchs
    batch_list = [i*batch_size for i in range(num_batchs+1)]
    batch_list[-1] = length
    batch_split_dict[lang] = batch_list
    print(lang, batch_list)

C++ [0, 12375, 24750, 37126]
Java [0, 3503, 7006, 10511]
Python [0, 8970, 17940, 26911]
C# [0, 1033, 2066, 3101]
C [0, 4141, 8282, 12424]


In [18]:
batch_programs_dict = {}
for lang in new_langs:
    batch_list = batch_split_dict[lang]
    batch_functions = []
    for bid in range(num_batchs):
        batch_dict = merged_filtered_dict[lang][batch_list[bid]:batch_list[bid+1]]
        functions = []
        for dic in batch_dict:
            functions.append(dic['function_notok'])
        batch_functions.append(functions)
    batch_programs_dict[lang] = batch_functions

In [27]:
lang1 = "Java"
lang2 = "Python"
batch_id = 0
src_codes = batch_programs_dict[lang1][batch_id]
tgt_codes = []
# infer with src and tgt
eval_examples, eval_features, eval_dataloader, model, tokenizer, args, decoder_sid = inference_prepro(
 lang1, lang2, model_type, device, src_codes, tgt_codes, None, tag, exp_suffix)

### Pre-sampling

In [5]:
data_name = "codenet"
is_eval = False
sample_size = 5
temperature = 0.5
tag = "all"
model_type = "plbart"
exp_suffix = "_translation_exec_function/" 

In [8]:
torch.cuda.empty_cache()

In [6]:
# get programs_dict
programs_dict = get_codenet_programs_dict(merged_filtered_dict)

In [7]:
# get import_str_dict
import_str_dict = {}
for lang in new_langs:
    all_imports, import_str = get_common_imports(lang, merged_filtered_dict)
    import_str_dict[lang] = import_str
import_str_dict["Java"] = java_imports_str
import_str_dict["C#"] = csharp_imports_str

#### Get preds of unpaired programs. Use script get_codenet_preds.py

In [ ]:
# get lang_pairs and preds
lang_pairs = []
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang2 != lang1:
            lang_pairs.append((lang1, lang2))
# small_programs_dict = {x:["haha"] for x in new_langs}
preds_lang_dict = get_preds_lang_dict_codenet(lang_pairs, model_type, device, programs_dict, 
                                      sample_size, temperature, data_name, tag, exp_suffix)

In [ ]:
def sampling_filtering_compilation(pids, eval_examples, eval_features, function_id_lang_dic,
                       reverse_map_dict, result_id_dict_gold, lang, model_type, model, tokenizer, args, device,
                       decoder_sid=None, is_eval=True, num_samples=5, temperature=0.5):
    selected_eval_examples, selected_eval_dataloader = get_eval_data_by_pid(eval_examples, eval_features, 
                                                               pids, reverse_map_dict, args.eval_batch_size)
    torch.cuda.empty_cache()
    preds, eval_result = generation_multiple(selected_eval_examples, 
                                             selected_eval_dataloader, 
                                             model, tokenizer, args, device, 
                                             decoder_sid, is_eval, num_samples, temperature)
    programs, program_id_dict, program_dict = prep_exec_hypo(preds, pids,
                                                             function_id_lang_dic, lang, model_type)
    lang_results = p_map(file_executors[lang], programs)
    result_id_dict, result_key_dict, error_type_dict = result_mapping(lang_results, program_id_dict, 
                                                                      pids, lang)
    buggy_pids, failed_test_pids, passed_hypo_dict = hypo_filtering(result_id_dict, 
                                                                    result_id_dict_gold, 
                                                                    result_key_dict, 
                                                                    program_dict)
    print(len(buggy_pids),len(failed_test_pids), len(passed_hypo_dict), len(pids))
    return buggy_pids, failed_test_pids, passed_hypo_dict

### Merge preds

In [ ]:
preds_lang_dict_all = {}
for lang in new_langs:
    dic_path = cached_path + "plbart_all_" + lang + "_codenet_preds_lang_dict.pkl"
    if not os.path.exists(dic_path):
        continue
    with open(dic_path, 'rb') as infile:
        preds_lang_dict_batch = pickle.load(infile)
        for k, v in preds_lang_dict_batch.items():
            print(lang, k)
            preds_lang_dict_all[k] = v
print(preds_lang_dict_all.keys())

In [5]:
plbart_sample_path = cached_path + 'plbart_codenet_preds_lang_dict.pkl'
with open(plbart_sample_path, 'rb') as infile:
     preds_lang_dict_plbart = pickle.load(infile)

In [7]:
codet5_sample_path = cached_path + 'codet5_codenet_preds_lang_dict.pkl'
with open(codet5_sample_path, 'rb') as infile:
     preds_lang_dict_codet5 = pickle.load(infile)

### Hypo filtering loop

In [ ]:
lang_pairs = [('Python', 'Java'), ('Python', 'C#'),
            ('Java', 'C#'), ('Java', 'C++'), ('Java', 'C'), 
            ('C#', 'Java'), ('C#', 'Python'), ('C#', 'C++'), ('C#', 'C')]
call_dict = {}
for lang1, lang2 in lang_pairs:
    print(lang1, lang2)
    preds = preds_lang_dict_all[(lang1, lang2)]
    new_preds = get_dedup_preds(preds)
    functions, function_id_dict = prep_exec_hypo_codenet(new_preds, lang1, lang2, 
                                                                 merged_filtered_dict, model_type)
    call_list = get_hypo_call_list(functions, lang2)
    call_dict[(lang1, lang2)] = [new_preds, functions, function_id_dict, call_list]
    with open(cached_path + "codenet_lang_pair_call_dict.pkl", 'wb') as outfile:
        pickle.dump(call_dict, outfile)

In [64]:
with open(cached_path + "plbart_codenet_lang_pair_call_dict.pkl", 'rb') as infile:
    call_dict = pickle.load(infile)

#### Get all the filtered hypos 

In [65]:
filtered_lang_dict = {}
for lang1, lang2 in call_dict.keys():
    new_preds, functions, function_id_dict, call_list = call_dict[(lang1, lang2)] 
    filtered_dict = get_compiled_hypos(call_list, function_id_dict, merged_filtered_dict)
    filtered_lang_dict[(lang1, lang2)] = filtered_dict

#### Separate pids that have filtered hypos and pids that don't

In [ ]:
empty_dict = {}
non_empty_dict = {}
for lang1, lang2 in filtered_lang_dict.keys():
    non_empty_dict[(lang1, lang2)] = []
    empty_dict[(lang1, lang2)] = []
    filtered_dict = filtered_lang_dict[(lang1, lang2)]
    for pid, inds in filtered_dict.items():
        if len(inds) > 0:
            non_empty_dict[(lang1, lang2)].append(pid)
        else:
            empty_dict[(lang1, lang2)].append(pid)
    print(lang1, lang2, len(non_empty_dict[(lang1, lang2)]))

### Check filtered hypo quality

In [101]:
# compare input output and check quality
# Quality is very good!
# numerics translation is not accurate.
lang1 = 'C++'
lang2 = 'Python'
new_preds, functions, function_id_dict, call_list = call_dict[(lang1, lang2)] 
filtered_dict = filtered_lang_dict[(lang1, lang2)]
src_codes = programs_dict[lang1]
src_codes_formatted = [x['function'] for x in merged_filtered_dict[lang1]]
non_empty_list = non_empty_dict[(lang1, lang2)]

In [ ]:
pid = 112
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang2 == lang1:
            continue
        new_preds, functions, function_id_dict, call_list = call_dict[(lang1, lang2)] 
        filtered_dict = filtered_lang_dict[(lang1, lang2)]
        src_codes = programs_dict[lang1]
        src_codes_formatted = [x['function'] for x in merged_filtered_dict[lang1]]
        non_empty_list = non_empty_dict[(lang1, lang2)]
        key = non_empty_list[-1]
        if len(filtered_dict[key]) > 0:
            print(lang1, lang2)
            print(detok_format(functions[filtered_dict[key][0]], file_detokenizers[lang2]))
            print(src_codes_formatted[key])

### Process filtered hypo for training data

In [146]:
lang1 = 'Java'
lang2 = 'Python'
new_preds, functions, function_id_dict, call_list = call_dict[(lang1, lang2)] 

In [158]:
for func in programs_dict["Java"]:
    if "J_T" in func:
        print(func)
        break

public void run(J_TOKEN.io.InputStream in,J_TOKEN.io.PrintStream out){ J_TOKEN.util.Scanner sc = new J_TOKEN.util.Scanner(in); int a, b, i, j, dig; for(;sc.hasNextInt();){ a = sc.nextInt(); b = sc.nextInt(); dig = caldig(a + b); System.out.println(dig); } sc.close(); } private static int caldig(int n){ int i; for(i = 1;i < 10;i++){ if(n / 10 == 0)return i; n = n / 10; } return 0; }


In [ ]:
# functions, notok_prepro(codestring, lang, is_plbart)
# remove empty lines (caused by tokenization)
# save into paired files; create map_file

In [133]:
from codenet_process_utils import *

In [ ]:
lang_pair_dict = get_lang_pair_dict(call_dict, merged_filtered_dict, programs_dict, False)


In [157]:
for lang_pair in lang_pair_dict.keys():
    lang1, lang2 = lang_pair
    new_src_codes, new_target_codes, new_pids = lang_pair_dict[lang_pair]
    if lang1 == "Java":
        for func in new_src_codes:
            if "J_T" in func:
                print("lang1", func)
                break

lang1 public static String calc(int[] tempInt){ J_TOKEN.util.Arrays.sort(tempInt); if((tempInt[0]*tempInt[0])+(tempInt[1]*tempInt[1])==(tempInt[2]*tempInt[2])){ return "YES"; }else{ return "NO"; } }


#### Check alignment

In [243]:
with open(cached_path + "codenet_src_hypo_pair_dict_plbart.pkl", 'rb') as infile:
    lang_pair_dict = pickle.load(infile)

In [244]:
lang_pair1 = ("Java", "Python")
lang_pair2 = ("Python", "Java")
src_codes1, target_codes1, pids1 = lang_pair_dict[lang_pair1]
src_codes2, target_codes2, pids2 = lang_pair_dict[lang_pair2]

In [298]:
print(lang_pair1)

lang_pair1 = ("Python", "C++")
src_codes1, target_codes1, pids1 = lang_pair_dict[lang_pair1]
lang1, lang2 = lang_pair1
key = 963
print(detok_format(src_codes1[key], file_detokenizers[lang1]))
print(detok_format(target_codes1[key], file_detokenizers[lang2]))

('Python', 'C++')
def check ( n , array ) :
    if 1.1 <= n :
        array [ 0 ] += 1
    elif 0.6 <= n < 1.1 :
        array [ 1 ] += 1
    elif 0.2 <= n < 0.6 :
        array [ 2 ] += 1
    else :
        array [ 3 ] += 1
void check ( int n , int array [ ] )
{
  if ( 1.0 <= n )
  array [ 0 ] ++ ;
  else if ( 1.0 <= n )
  array [ 1 ] ++ ;
  else if ( 0.2 <= n )
  array [ 2 ] ++ ;
  else
  array [ 3 ] ++ ;
}


In [131]:
merged_lang_pair_dict = get_merged_lang_pair_dict(lang_pair_dict)

('C++', 'Java') 17056
('C++', 'Python') 42234
('C++', 'C#') 15599
('C++', 'C') 858
('Java', 'Python') 16851
('Java', 'C#') 3787
('Java', 'C') 786
('Python', 'C#') 9629
('Python', 'C') 9549
('C#', 'C') 367


In [31]:
# ('C++', 'Java') 17056
# ('C++', 'Python') 41701
# ('C++', 'C#') 15599
# ('Java', 'Python') 16457
# ('Python', 'C#') 9546
# ('Java', 'C#') 3787
# ('Python', 'C') 8731


'/home/mingzhu/CodeModel/CodeGen_cwd/codenet_function_pairs/'

#### Split into train-val-test

In [79]:
# Any constraints?
# Simple. Just split at problem level
all_problem_ids = get_all_problem_ids(merged_lang_pair_dict)

C++ Java
C++ Python
C++ C#
C++ C
Java Python
Java C#
Java C
Python C#
Python C
C# C
C++ 35303
Java 9212
Python 9233
C# 2852
C 8480


In [86]:
all_problem_ids_list = list(all_problem_ids)
train_ratio = 0.85
val_ratio = 0.05
test_ratio = 0.1
num_problems = len(all_problem_ids_list)
train_num = int(train_ratio*num_problems)
test_num = int(test_ratio*num_problems)
train_proids = all_problem_ids_list[:train_num]
test_proids = all_problem_ids_list[num_problems-test_num:]
val_proids = all_problem_ids_list[train_num:num_problems-test_num]
codenet_hypo_split_dict = {'train':train_proids, 'test':test_proids, 'val':val_proids}

In [92]:
with open(cached_path + "codenet_hypo_split_dict.json", 'w') as outfile:
    json.dump(codenet_hypo_split_dict, outfile)

In [ ]:
with open(cached_path + "codenet_hypo_split_dict.json") as infile:
    codenet_hypo_split_dict = json.load(infile)

In [134]:
split_lang_pair_dict = get_split_lang_pair_dict(merged_lang_pair_dict, merged_filtered_dict, 
                                                codenet_hypo_split_dict)

C++ Java
C++ Java [13706, 375, 2975]
C++ Python
C++ Python [35250, 1393, 5591]
C++ C#
C++ C# [12531, 427, 2641]
C++ C
C++ C [661, 8, 189]
Java Python
Java Python [13746, 644, 2461]
Java C#
Java C# [2908, 132, 747]
Java C
Java C [522, 11, 253]
Python C#
Python C# [7922, 271, 1436]
Python C
Python C [7924, 315, 1310]
C# C
C# C [261, 7, 99]


#### Write into parallel files

In [144]:
size_dict = {}
for lang1, lang2 in merged_lang_pair_dict.keys():
    size_dict[(lang1, lang2)] = 0
    src_codes, target_codes, pids = merged_lang_pair_dict[(lang1, lang2)]
    for tag in tags:
        tag_indices = split_lang_pair_dict[(lang1, lang2)][tag]
        for tag_i in tag_indices:
            pid = pids[tag_i]
            src_code = src_codes[tag_i]
            target_code = target_codes[tag_i]
            if len(src_code) > 1000:
                size_dict[(lang1, lang2)] += 1
    print(lang1, lang2, size_dict[(lang1, lang2)] )

C++ Java 242
C++ Python 2353
C++ C# 228
C++ C 0
Java Python 903
Java C# 22
Java C 1
Python C# 659
Python C 271
C# C 0


In [136]:
codenet_pair_path = codenet_processed_data_path + "codenet_function_pairs_non_plbart/"
write_codenet_pairdata(merged_lang_pair_dict, split_lang_pair_dict, codenet_pair_path)

#### Check alignment again

In [ ]:
for lang1, lang2 in merged_lang_pair_dict.keys():
    src_codes, target_codes, pids = merged_lang_pair_dict[(lang1, lang2)]
    for tag in tags:
        tag_indices = split_lang_pair_dict[(lang1, lang2)][tag]
        tag_i = tag_indices[-1]
        src_code = src_codes[tag_i]
        target_code = target_codes[tag_i]
        print(lang1, lang2)
        print(src_code)
        print(target_code)

### Create non-plbart dataset

In [163]:
from codenet_process_utils import *

In [ ]:
is_plbart = True
merged_filtered_dict = get_prepro_filtered_dict(merged_filtered_dict, is_plbart)
programs_dict = get_codenet_programs_dict(merged_filtered_dict)
with open(cached_path + "codet5_codenet_lang_pair_call_dict.pkl", 'rb') as infile:
    call_dict = pickle.load(infile)
# lang_pair_dict = get_lang_pair_dict(call_dict, merged_filtered_dict, programs_dict, is_plbart)
# with open(cached_path + "codet5_codenet_src_hypo_pair_dict_plbart.pkl", 'wb') as outfile:
#     pickle.dump(lang_pair_dict, outfile)
with open(cached_path + "codet5_codenet_src_hypo_pair_dict_plbart.pkl", 'rb') as infile:
    lang_pair_dict = pickle.load(infile)

In [9]:
with open(cached_path + "codet5_codenet_lang_pair_call_dict.pkl", 'rb') as infile:
    call_dict = pickle.load(infile)

In [207]:
for lang_pair, lists in lang_pair_dict.items():
    print(lang_pair, len(lists[0]))

('Java', 'C') 1251
('Python', 'C') 4730
('C#', 'C') 214
('C++', 'C') 13451
('C', 'Java') 5844
('C', 'Python') 7867
('C', 'C#') 5524
('C', 'C++') 8765


In [208]:
merged_lang_pair_dict = get_merged_lang_pair_dict(lang_pair_dict)
codenet_pair_path = codenet_processed_data_path + "codet5_codenet_function_pairs/"
if not os.path.exists(codenet_pair_path):
    os.mkdir(codenet_pair_path)
split_lang_pair_dict = get_split_lang_pair_dict(merged_lang_pair_dict, merged_filtered_dict, 
                                                codenet_hypo_split_dict)
write_codenet_pairdata(merged_lang_pair_dict, split_lang_pair_dict, codenet_pair_path)

('C++', 'Java') 0
('C++', 'Python') 0
('C++', 'C#') 0
('C++', 'C') 22216
('Java', 'Python') 0
('Java', 'C#') 0
('Java', 'C') 7095
('Python', 'C#') 0
('Python', 'C') 12597
('C#', 'C') 5738
C++ Java
C++ Java [0, 0, 0]
C++ Python
C++ Python [0, 0, 0]
C++ C#
C++ C# [0, 0, 0]
C++ C
C++ C [17940, 507, 3769]
Java Python
Java Python [0, 0, 0]
Java C#
Java C# [0, 0, 0]
Java C
Java C [5599, 117, 1379]
Python C#
Python C# [0, 0, 0]
Python C
Python C [10282, 362, 1953]
C# C
C# C [4598, 87, 1053]


In [166]:
is_plbart = True

merged_filtered_dict = get_prepro_filtered_dict(merged_filtered_dict, is_plbart)
programs_dict = get_codenet_programs_dict(merged_filtered_dict)
with open(cached_path + "codenet_lang_pair_call_dict_new.pkl", 'rb') as infile:
    call_dict = pickle.load(infile)
# lang_pair_dict = get_lang_pair_dict(call_dict, merged_filtered_dict, programs_dict, is_plbart)
with open(cached_path + "codenet_src_hypo_pair_dict_plbart.pkl", 'rb') as infile:
    lang_pair_dict = pickle.load(infile)
merged_lang_pair_dict = get_merged_lang_pair_dict(lang_pair_dict)
codenet_pair_path = codenet_processed_data_path + "codenet_function_pairs/"
write_codenet_pairdata(merged_lang_pair_dict, split_lang_pair_dict, codenet_pair_path)

('C++', 'Java') 17056
('C++', 'Python') 42234
('C++', 'C#') 15599
('C++', 'C') 858
('Java', 'Python') 16851
('Java', 'C#') 3787
('Java', 'C') 786
('Python', 'C#') 9629
('Python', 'C') 9549
('C#', 'C') 367


#### Create a smaller dataset

In [184]:
is_plbart = True
merged_filtered_dict = get_prepro_filtered_dict(merged_filtered_dict, is_plbart)
programs_dict = get_codenet_programs_dict(merged_filtered_dict)
with open(cached_path + "codenet_src_hypo_pair_dict_plbart.pkl", 'rb') as infile:
    lang_pair_dict = pickle.load(infile)
# merged_lang_pair_dict = get_merged_lang_pair_dict(lang_pair_dict)
# codenet_pair_path = codenet_processed_data_path + "codenet_function_pairs_small/"
# write_codenet_pairdata(merged_lang_pair_dict, split_lang_pair_dict, codenet_pair_path)

In [ ]:
merged_lang_pair_dict = {}
iterated_set = set()
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang2 == lang1:
            continue
        lang_pair1 = (lang1, lang2)
        if lang_pair1 in iterated_set:
            continue
        lang_pair2 = (lang2, lang1)
        iterated_set.add(lang_pair1)
        iterated_set.add(lang_pair2)
        
        src_codes1, target_codes1, pids1 = [], [], []
        src_codes2, target_codes2, pids2 = [], [], []
        if lang_pair1 in lang_pair_dict:
            src_codes1, target_codes1, pids1 = lang_pair_dict[lang_pair1]
        if lang_pair2 in lang_pair_dict:
            src_codes2, target_codes2, pids2 = lang_pair_dict[lang_pair2]
        src_codes, target_codes, pids = src_codes1, target_codes1, pids1
        pids = [lang1 + "-" + str(x) for x in pids1]
        if len(src_codes1) > len(src_codes2):
            src_codes, target_codes, pids = target_codes2, src_codes2, pids2
            pids = [lang2 + "-" + str(x) for x in pids2]
        merged_lang_pair_dict[lang_pair1] = [src_codes, target_codes, pids]
        print(lang_pair1, len(pids))


In [ ]:
split_lang_pair_dict = get_split_lang_pair_dict(merged_lang_pair_dict, merged_filtered_dict, 
                                                codenet_hypo_split_dict)
codenet_pair_path = codenet_processed_data_path + "codenet_function_pairs_small/"
if not os.path.exists(codenet_pair_path):
    os.mkdir(codenet_pair_path)
write_codenet_pairdata(merged_lang_pair_dict, split_lang_pair_dict, codenet_pair_path)

### Eval model trained on the new dataset

In [ ]:
# Use what to eval? if use xlcost, it's not fair because there's domain shift.
# Should eval on leetcode. 

### Resampling for pids that doesn't have filtered hypo

In [ ]:
# resamples for pids in empty_dict

In [30]:
lang1 = "C#"
lang2 = "C++"
preds = preds_lang_dict_all[(lang1, lang2)][:100]
new_preds = get_dedup_preds(preds)
functions, function_id_dict = prep_exec_hypo_codenet(new_preds, lang1, lang2, 
                                                            merged_filtered_dict, model_type)
# call_list = get_hypo_call_list(functions, lang2)
# filtered_dict = get_compiled_hypos(call_list, function_id_dict, merged_filtered_dict)

100%|██████████| 100/100 [00:01<00:00, 76.87it/s]


In [32]:
call_list = get_hypo_call_list(functions, lang2)


C++ error 348 0.8169014084507042
C++ timeout 0 0.0
C++ empty 78 0.18309859154929578
C++ other 0 0.0
C++ good 0 0.0


In [ ]:
filtered_dict = get_compiled_hypos(call_list, function_id_dict, merged_filtered_dict)

In [ ]:
for i, (error_key, error)  in enumerate(zip(result_keys, processed_results)):
    if error_key == "error":
        if "Compilation failed" in error:
            print(i, error)
            break

### Codenet data preprocessing

#### Get input stats

In [ ]:
num_batch = 41
code_lang_dict_list = []
len_dict = {}
for i in tqdm(range(num_batch)):
    codedict_path = cached_path + 'codenet/codenet_codedict_' + str(i) + '.json'
    if os.path.exists(codedict_path):
        with open(cached_path + 'codenet/codenet_problems_dict_' + str(i) + '.json') as infile:
            codenet_problems_dict_batch = json.load(infile)
        programs_dict, programs_idx_dict, program_id_dict = get_codenet_programs(
                                                                codenet_problems_dict_batch, new_langs)
        len_dict[i] = {}
        for lang in new_langs:
            len_dict[i][lang] = len(programs_dict[lang])
            print(lang, len(programs_dict[lang]))
#         code_lang_dict = get_codenet_code_dict(programs_dict, programs_idx_dict, program_id_dict, 
#                                            codenet_problems_dict_batch)

        with open(codedict_path) as infile:
            code_lang_dict = json.load(infile)
        code_lang_dict_list.append(code_lang_dict)
#         for lang in new_langs:
#             print(lang, len(code_lang_dict[lang]))
#         break

In [199]:
len_lang_dict = {x:0 for x in new_langs}
for i, dic in len_dict.items():
    for lang in new_langs:
        len_lang_dict[lang] += dic[lang]

In [200]:
# raw input programs
print(len_lang_dict)

{'C++': 525822, 'Java': 87482, 'Python': 132316, 'C#': 19362, 'C': 158246}


In [203]:
# programs with at least one function
count_lang_dict = {x:0 for x in new_langs}
for code_lang_dict in code_lang_dict_list:
    for lang in new_langs:
        dics = code_lang_dict[lang]
        #TODO. Count dics have more than one func
        for dic in dics:
            if len(dic['functions']) > 0:
                count_lang_dict[lang] += 1
count_lang_dict
    

{'C++': 251732, 'Java': 30709, 'Python': 28830, 'C#': 7651, 'C': 50265}

#### Get codedicts

In [ ]:
num_batch = 41
code_lang_dict_list = []
for i in tqdm(range(26, num_batch)):
    with open(cached_path + 'codenet_problems_dict_' + str(i) + '.json') as infile:
        codenet_problems_dict_batch = json.load(infile)
    programs_dict, programs_idx_dict, program_id_dict = get_codenet_programs(
                                                            codenet_problems_dict_batch, new_langs)
    code_lang_dict = get_codenet_code_dict(programs_dict, programs_idx_dict, program_id_dict, 
                                       codenet_problems_dict_batch)
    with open(cached_path + 'codenet_codedict_' + str(i) + '.json', 'w') as outfile:
        json.dump(code_lang_dict, outfile)
    code_lang_dict_list.append(code_lang_dict)

#### Filter out programs that compiles

In [ ]:
call_dict_list = []
filtered_dict_list = []
for code_lang_dict in tqdm(code_lang_dict_list):
    func_id_dict, program_dict, imports_dict = get_nonempty_functions(code_lang_dict, new_langs)
    call_dict = get_codenet_call_dict(program_dict, imports_dict, new_langs)
    call_dict_list.append(call_dict)
    filtered_dict = get_compiled_functions(call_dict, func_id_dict, imports_dict, program_dict, 
                                       code_lang_dict)
    filtered_dict_list.append(filtered_dict)

#### Merge into one dataset

In [ ]:
# merge all the filtered programs
merge_filtered_dict(num_dicts)

In [ ]:
# remove langs other than the 7 langs
fns = os.listdir(codenet_data_path)
for fn in fns:
    lang_path = codenet_data_path + fn + '/'
    lang_fns = os.listdir(lang_path)
    for lang_fn in lang_fns:
        print(lang_fn)
        if lang_fn not in langs:
            shutil.rmtree(lang_path + lang_fn)